In [ ]:
# To mount the colab notebook to drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Module Imports
import tensorflow as tf
import numpy as np

In [ ]:
img = tf.keras.preprocessing.image.load_img('Group_Project_Data/Train/Real/img_0.png')
img

# Define the image directory path
image_dir_train = "Group_Project_Data/Train"
image_dir_valid = "Group_Project_Data/Valid"

# Define the batch size
batch_size = 32

# Define the image size
img_size = (64, 64)


In [ ]:
# Create the dataset using the image directory path
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=image_dir_train,
    labels="inferred",
    label_mode="binary",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    color_mode='grayscale',
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=image_dir_valid,
    labels="inferred",
    label_mode="binary",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    color_mode='grayscale',
)

In [ ]:
# Print the class names
class_names = train_dataset.class_names
print(class_names)


In [ ]:
# Load the training data 
x_train = []
y_train = []
for images, labels in train_dataset:
    x_train.append(images.numpy())
    y_train.append(labels.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)



In [ ]:
# Load the validation data 
x_val = []
y_val = []
for images, labels in val_dataset:
    x_val.append(images.numpy())
    y_val.append(labels.numpy())

x_val = np.concatenate(x_val, axis=0)
y_val = np.concatenate(y_val, axis=0)




In [ ]:
# Rescale the pixel values
x_train = x_train/255.
x_val = x_val/255.

In [ ]:
# Define the model architecture with L2 regularization
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), kernel_regularizer=l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(layers.Dense(1, activation='sigmoid'))

# Print the summary of the model architecture
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, batch_size =128, epochs=10)
print(history.history.keys)

In [ ]:
# Plot for Model Loss
plt.figure()
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend(loc=0)
plt.title('Model Loss')

In [ ]:
# Plot for Model Accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.legend(loc=0)
plt.title('Model Accuracy')

plt.show()

In [ ]:
# Testing the model to predict 
y_pred = model.predict(x_val)
y_pred_label = np.round(y_pred)

# Plotting images with labels telling the truth and predicted values with some 
# testing images for each class
f, ax = plt.subplots(5,5, figsize=(10, 12))
ax = ax.flatten()

# Looping the selected images from each class to show on the plot
for i in range(25):
    ax[i].imshow(x_val[i,:,:])
    ax[i].set_title(f'Truth = {class_names[int(y_val[i])]}\nPred = {class_names[int(y_pred_label[i])]}')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.show()

print(len(x_val))

In [ ]:
# Saving model

model.save('/content/Group_43_Model_Trained')

In [ ]:
# Test saved model

loaded_model = tf.keras.models.load_model('/content/Group_43_Model_Trained')
loaded_model.summary()